<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [1]:
;pwd

/home/ubuntu/work/Github/JWAS.jl/docs/notebooks_v0.3


In [1]:
include("/home/ubuntu/work/Github/JWAS.jl/src/JWAS.jl")

JWAS

In [2]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [3]:
phenofile  = Datasets.dataset("example","phenotypes_ssbr.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


Finished!


In [4]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [7]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [8]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [9]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [10]:
G1 = 1.0
G2 = eye(2)
set_random(model1,"x2",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [11]:
G3 =1.0
add_genotypes(model1,genofile,G3,separator=',');

5 markers on 7 individuals were added.


In [12]:
JWAS.outputEBV(model1,["a1","a2","a3"]);

Estimated breeding values and prediction error variances will be included in the output.


In [13]:
outputMCMCsamples(model1,"x2")

In [24]:
outputMCMCsamples(model1,"J")

In [15]:
fieldnames(model1)

32-element Array{Symbol,1}:
 :nModels         
 :modelVec        
 :modelTerms      
 :modelTermDict   
 :lhsVec          
 :covVec          
 :X               
 :ySparse         
 :obsID           
 :mmeLhs          
 :mmeRhs          
 :pedTrmVec       
 :ped             
 ⋮                
 :resVar          
 :ROld            
 :RNew            
 :M               
 :mmePos          
 :samples4R       
 :samples4G       
 :outputSamplesVec
 :df              
 :output_ID       
 :output_genotypes
 :output_X        

In [20]:
JWAS.SSBRrun(model1,pedigree,phenotypes)

In [23]:
?JWAS.SSBRrun

No documentation found.

`JWAS.SSBRrun` is a `Function`.

```
# 1 method for generic function "SSBRrun":
SSBRrun(mme, ped::JWAS.PedModule.Pedigree, df) in JWAS at /home/ubuntu/work/Github/JWAS.jl/src/1.JWAS/src/MCMC/../SSBR/SSBR.jl:2
```


In [26]:
model1.outputSamplesVec

1-element Array{JWAS.MCMCSamples,1}:
 JWAS.MCMCSamples(JWAS.ModelTerm(1, "1:x2", 1, Symbol[:x2], AbstractString[], Float64[], 0, Any[], 0, 0×0 SparseMatrixCSC{Float64,Int64} with 0 stored entries), [6.94573e-310])

In [22]:
model1.modelTerms

8-element Array{JWAS.ModelTerm,1}:
 JWAS.ModelTerm(1, "1:intercept", 1, Symbol[:intercept], AbstractString[], Float64[], 0, Any[], 0, 0×0 SparseMatrixCSC{Float64,Int64} with 0 stored entries)               
 JWAS.ModelTerm(1, "1:x1*x3", 2, Symbol[:x1, :x3], AbstractString[], Float64[], 0, Any[], 0, 0×0 SparseMatrixCSC{Float64,Int64} with 0 stored entries)                     
 JWAS.ModelTerm(1, "1:x2", 1, Symbol[:x2], AbstractString[], Float64[], 0, Any[], 0, 0×0 SparseMatrixCSC{Float64,Int64} with 0 stored entries)                             
 JWAS.ModelTerm(1, "1:x3", 1, Symbol[:x3], AbstractString[], Float64[], 0, Any[], 0, 0×0 SparseMatrixCSC{Float64,Int64} with 0 stored entries)                             
 JWAS.ModelTerm(1, "1:ID", 1, Symbol[:ID], AbstractString[], Float64[], 0, Any[], 0, 0×0 SparseMatrixCSC{Float64,Int64} with 0 stored entries)                             
 JWAS.ModelTerm(1, "1:dam", 1, Symbol[:dam], AbstractString[], Float64[], 0, Any[], 0, 0×0 SparseMatrixCS

<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [12]:
outputMCMCsamples(model1,"x2")
out1=runMCMC(model1,phenotypes,methods="RR-BLUP",single_step_analysis=true,pedigree=pedigree,chain_length=5000,output_samples_frequency=100);


The prior for marker effects variance is calculated from 
the genetic variance and π. The prior for the marker effects variance 
is: 0.492462



A Linear Mixed Model was build using model equations:

y1 = intercept + x1*x3 + x2 + x3 + ID + dam

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1*x3           interaction  fixed                2
x2              factor       random               2
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
ϵ               factor       random               5
J               covariate    fixed                1

MCMC Information:

methods                                     RR-BLUP
chain_length                                   5000
burnin                                            0
estimatePi                                    false
starting_value                        

running MCMC for RR-BLUP...100%|████████████████████████| Time: 0:00:01


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [13]:
keys(out1)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of marker effects"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"

In [14]:
out1["EBV_y1"]

3×2 Array{Any,2}:
 "a1"  -2.88462
 "a2"  -1.71113
 "a3"  -3.8475 

In [15]:
out1["Posterior mean of marker effects"]

5×2 Array{Any,2}:
 "m1"  -0.0586367
 "m2"  -0.22454  
 "m3"   0.390271 
 "m4"  -0.297191 
 "m5"  -0.141419 

In [16]:
out1["Posterior mean of location parameters"]

,Trait,Effect,Level,Estimate
1,1,intercept,intercept,-17.2235
2,1,x1*x3,x1 * m,-3.26901
3,1,x1*x3,x1 * f,0.502404
4,1,x2,2,0.139786
5,1,x2,1,-0.152847
6,1,x3,m,21.6859
7,1,x3,f,18.4122
8,1,ID,a12,0.284532
9,1,ID,a10,-0.336192
10,1,ID,a11,-0.0266158


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multi-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [17]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [18]:
R      = eye(3)
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [19]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [20]:
G1 = eye(2)
G2 = eye(4)
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

INFO: x2 is not found in model equation 1.
INFO: dam is not found in model equation 2.
INFO: dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [21]:
G3 = eye(3)
add_genotypes(model2,genofile,G3,separator=',');

5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [22]:
out2=runMCMC(model2,phenotypes,methods="RR-BLUP",single_step_analysis=true,pedigree=pedigree,chain_length=5000,output_samples_frequency=100);

INFO: Pi (Π) is not provided.
INFO: Pi (Π) is generated assuming all markers have effects on all traits.



The prior for marker effects covariance matrix is calculated from 
genetic covariance matrix and Π. The prior for the marker effects 
covariance matrix is: 

 0.492462  0.0       0.0     
 0.0       0.492462  0.0     
 0.0       0.0       0.492462


A Linear Mixed Model was build using model equations:

y1 = intercept + x1 + x3 + ID + dam
y2 = intercept + x1 + x2 + x3 + ID
y3 = intercept + x1 + x1*x3 + x2 + ID

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1              covariate    fixed                1
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
x2              factor       random               2
x1*x3           interaction  fixed                2
ϵ               factor       random               5
J               covariate    fixed                1

MCMC Information:

methods                 

running MCMC for RR-BLUP...100%|████████████████████████| Time: 0:00:04


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [23]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 9 entries. Keys:
  "EBV_y2"
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "EBV_y3"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"

In [24]:
out2["EBV_y2"]

8×2 Array{Any,2}:
 "a1"   -3.87823
 "a2"   -1.25862
 "a3"   -3.37202
 "a4"   -3.08974
 "a5"   -3.40619
 "a6"   -3.92451
 "a10"  -2.9702 
 "a12"  -4.0199 

In [25]:
out2["Posterior mean of marker effects"]

3-element Array{Any,1}:
 Any["m1" 0.0915618; "m2" -0.320541; … ; "m4" -0.219831; "m5" 0.120427]
 Any["m1" 0.0891959; "m2" 0.0282597; … ; "m4" 0.147042; "m5" 0.163463] 
 Any["m1" -0.187986; "m2" 0.15559; … ; "m4" 0.169466; "m5" 0.0578627]  

In [28]:
out2["Posterior mean of location parameters"][end,:]

,Trait,Effect,Level,Estimate
1,3,J,J,2.19746


In [31]:
model1.outputSamplesVec

1-element Array{JWAS.MCMCSamples,1}:
 JWAS.MCMCSamples(JWAS.ModelTerm(1, "1:x2", 1, Symbol[:x2], AbstractString["2", "1", "2", "1", "2", "2", "2", "1"], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 2, Any["2", "1"], 4, 
  [1, 1]  =  1.0
  [3, 1]  =  1.0
  [5, 1]  =  1.0
  [6, 1]  =  1.0
  [7, 1]  =  1.0
  [8, 1]  =  0.0
  [1, 2]  =  0.0
  [2, 2]  =  1.0
  [4, 2]  =  1.0
  [8, 2]  =  1.0), [-0.064095 0.307676; 0.0526382 -0.0132428; … ; 2.33262 1.67313; 0.507056 -0.1388])

In [38]:
model2.M

JWAS.Genotypes(AbstractString["a1", "a2", "a3", "a4", "a5", "a6", "a10", "a12"], SubString{String}["m1", "m2", "m3", "m4", "m5"], 7, 5, [0.642857 0.785714 … 0.428571 0.214286], 2.030612244897959, true, [-0.285714 0.428571 … 0.142857 -0.428571; 0.357143 -0.285714 … 0.0714286 0.285714; … ; -0.785714 -0.0714286 … 0.642857 0.571429; -0.0357143 -0.0714286 … 0.142857 0.0714286], [0.377801 -0.0527726 0.128327; -0.0527726 0.441608 0.194167; 0.128327 0.194167 0.295799], true)